# Developing JuliaFEM

Author(s): Jukka Aho

**Abstract**: Developer notes. In this notebook we give general guidelines how to implement things to JuliaFEM.

In short, we have

**Element**: object which holds basis functions and fields. One is able to interpolate fields using element. Introduction of new basis functions needs new elements. Typical elements: Lagrange elements, hierarchical elements, etc.

**Equation**: object which defines some field equation which needs to be solve. One element can have several equations but not vice versa. Typical equations: Poisson equation $\Delta u = f$, elasticity equation $\nabla \cdot \sigma = f$, etc.

**Problem**: object which maps equations to elements. For example, HeatProblem which maps Poisson equation to Lagrange elements or ElasticityProblem which maps elasticity equation to Lagrange elements.

**Solver**: object which takes one or more problems, solves them using some strategy (iterative methods, multigrid, direct methods, ...) and updates corresponding fields to elements.

Moreover, we have (will have) **`test_element`**, **`test_equation`**, **`test_problem`** and **`test_solver`** which can be used to test that implementation has all necessary things defined.

In [1]:
using Logging
using FactCheck
Logging.configure(level=DEBUG)

Logger(root,DEBUG,PipeEndpoint(open, 0 bytes waiting),root)

## Developing own element

Element contains basis functions and one or several fieldsets.

Minimum requirements for element:
- subclass it from Element, unless want to implement everything by youself
- define basis and partial derivatives of it, because we need to interpolate over it
- give connectivity information, how this element is connected to other elements

Test the element using ``test_element`` function. It it passes, the element implementation should be fine.

In [2]:
using JuliaFEM: Element, Field, FieldSet, Basis

Here's the basic implementation for element:

In [3]:
type MyQuad4 <: Element
    connectivity :: Array{Int, 1}
    basis :: Basis
    fields :: Dict{Symbol, FieldSet}
end

Default constructor takes only connectivity information as input argument. Most important thing here is to define `Basis` which is used to interpolate fields.

In [4]:
function MyQuad4(connectivity)
    h(xi) = [
        (1-xi[1])*(1-xi[2])/4
        (1+xi[1])*(1-xi[2])/4
        (1+xi[1])*(1+xi[2])/4
        (1-xi[1])*(1+xi[2])/4]
    dh(xi) = [
        -(1-xi[2])/4.0   -(1-xi[1])/4.0
         (1-xi[2])/4.0   -(1+xi[1])/4.0
         (1+xi[2])/4.0    (1+xi[1])/4.0
        -(1+xi[2])/4.0    (1-xi[1])/4.0]
    basis = Basis(h, dh)
    MyQuad4(connectivity, basis, Dict())
end

MyQuad4

Also some basic information like number of basis functions and element dimension is needed.

In [5]:
JuliaFEM.get_number_of_basis_functions(el::Type{MyQuad4}) = 4
JuliaFEM.get_element_dimension(el::Type{MyQuad4}) = 2

get_element_dimension (generic function with 7 methods)

Next we check that everything is well defined:

In [6]:
using JuliaFEM: test_element
test_element(MyQuad4)

09-Oct 01:41:19:INFO:root:Testing element MyQuad4
09-Oct 01:41:19:INFO:root:number of basis functions in this element: 4
09-Oct 01:41:19:INFO:root:Initializing element
09-Oct 01:41:19:INFO:root:Element dimension: 2
09-Oct 01:41:19:INFO:root:Creating new scalar field JuliaFEM.Field{Array{Int64,1}}(0.0,1,[1,2,3,4])
09-Oct 01:41:19:INFO:root:Pushing field to element.
09-Oct 01:41:19:INFO:root:Interpolating scalar field at [0.0,0.0]
09-Oct 01:41:19:INFO:root:Value: 2.5


PipeEndpoint(open, 0 bytes waiting)

If `test_element` passes, elements *interface* should be well defined. After building element, one can interpolate things in it. Here we create three new fieldsets `temperature`, `geometry` and `heat coefficient`, add some values for them in time $t=0.0$ and $t=1.0$ and interpolate:

In [7]:
using JuliaFEM: new_fieldset!, add_field!, interpolate, dinterpolate
el1 = MyQuad4([1, 2, 3, 4])
new_fieldset!(el1, "temperature")
add_field!(el1, "temperature", Field(0.0, [0.0, 0.0, 0.0, 0.0]))
add_field!(el1, "temperature", Field(1.0, [1.0, 2.0, 3.0, 4.0]))
new_fieldset!(el1, "geometry")
add_field!(el1, "geometry", Field(0.0, Vector[[0.0,0.0,0.0], [10.0,0.0,0.0], [10.0,1.0,0.0], [0.0,1.0,0.0]]))
new_fieldset!(el1, "heat coefficient")
add_field!(el1, "heat coefficient", Field(0.0, 2))
add_field!(el1, "heat coefficient", Field(1.0, 3))

2-element Array{JuliaFEM.Field{T},1}:
 JuliaFEM.Field{Int64}(0.0,1,2)
 JuliaFEM.Field{Int64}(1.0,1,3)

09-Oct 01:41:19:INFO:root:Element MyQuad4 passed tests.


In [8]:
# temperature at the middle poinf of the element, 1/4*(1+2+3+4) at t=0.5
interpolate(el1, "temperature", [0.0, 0.0], 0.5)

1.25

In [9]:
# geometry midpoint of element
interpolate(el1, "geometry", [0.0, 0.0], 0.0)

3-element Array{Float64,1}:
 5.0
 0.5
 0.0

In [10]:
# interpolate derivatives works too
dinterpolate(el1, "geometry", [0.0, 0.0], 0.0)

3x2 Array{Float64,2}:
 5.0  0.0
 0.0  0.5
 0.0  0.0

In [11]:
# interpolating scalar -> scalar.
interpolate(el1, "heat coefficient", [0.0, 0.0], 0.5)

2.5

In [12]:
interpolate(el1, "heat coefficient", [0.0, 0.0], Inf)

3

### Summary of developing own elements

Element itself if not calculating anything but only stores fields and basis functions so that the fields can be interpolated. We will provide command `test_element` which will ensure that everything necessary is defined. While lot of things needs to be defined, by subclassing from `Element` most of these are already defined, thanks to multiple dispatch.

## Developing own equation

Let's consider a Laplace equation
\begin{align}
\Delta{u} &= 0 && \text{on } \Omega \\
\frac{\partial u}{\partial n} &= g && \text{on } \Gamma_{\mathrm{N}}
\end{align}

Weak form is, find $u\in\mathcal{U}$ such that
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x = \int_{\Gamma_{\mathrm{N}}}g v\,\mathrm{d}s \quad \forall v\in\mathcal{V}.
\end{equation}

Minimum requirements for equation: 
- subclass from Equation, if not want to implement from scratch
- it needs to have lhs and rhs functions
- provide the name of the unknown field variable trying to solve
- default constructor takes the element as input argument

Now we have function `test_equation`, which we can use to test that everything is working as expected. 

Again thanks to multiple dispatch, you are free to code your weak form however you want as long as it returns lhs and rhs sides for element dofs. This kind of freedom gives good opportunities to wrap e.g. Fortran code from some other projects. And again we have some suggestions ad following these ideas you get a lot of stuff for free. First we look the left hand side of the equation, that is,
\begin{equation}
    \int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x
\end{equation}

In [13]:
using JuliaFEM: Equation, IntegrationPoint, Quad4

abstract Heat <: Equation

get_unknown_field_name(eq::Heat) = symbol("temperature")

get_unknown_field_name (generic function with 1 method)

Our basic data type often looks something like this:

In [14]:
"""
Diffusive heat transfer for 4-node bilinear element.
"""
type DC2D4 <: Heat
    element :: Quad4
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

We must provide default constructor which takes element as input argument:

In [15]:
function DC2D4(el::Quad4)
    integration_points = [
        IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0),
        IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0)]
    new_fieldset!(el, "temperature") # assign new field "temperature" to element
    DC2D4(el, integration_points, [])
end

DC2D4

Now the actual implementation for $\int_{\Omega}\nabla u\cdot\nabla v\,\mathrm{d}x$:

In [16]:
using JuliaFEM: get_element, get_dbasisdX, has_lhs, has_rhs

"""
Left hand side defined in integration point
"""
function JuliaFEM.get_lhs(eq::DC2D4, ip, t)
    el = get_element(eq)
    dNdX = get_dbasisdX(el, ip.xi, t)
    hc = interpolate(el, "temperature thermal conductivity", ip.xi, t)
    return dNdX*hc*dNdX'
end
JuliaFEM.has_lhs(eq::DC2D4) = true

has_lhs (generic function with 2 methods)

And that's it. If we want to play with this formulation, we must create element and assign this equation for it:

In [17]:
using JuliaFEM: integrate, integrate_lhs, integrate_rhs, new_fieldset!, add_field!
el = Quad4([1, 2, 3, 4])
new_fieldset!(el, "geometry")
add_field!(el, "geometry", Field(0.0, Vector[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]]))
new_fieldset!(el, "temperature thermal conductivity")
add_field!(el, "temperature thermal conductivity", Field(0.0, 6.0))
eq = DC2D4(el)
integrate_lhs(eq, 1.0)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [18]:
JuliaFEM.has_lhs(eq)

true

If rhs or lhs is not defined, integration returns nothing.

In [19]:
isa(integrate_rhs(eq, 1.0), Void)

true

Next heat flux on boundary:

In [20]:
using JuliaFEM: get_basis, Seg2

"""
Diffusive heat transfer for 2-node linear segment.
"""
type DC2D2 <: Heat
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
end

function DC2D2(el::Seg2)
    integration_points = [
        IntegrationPoint([0], 2.0)]
    new_fieldset!(el, "temperature")
    DC2D2(el, integration_points, [])
end

"""
Right hand side defined in integration point
"""
function JuliaFEM.get_rhs(eq::DC2D2, ip, t)
    el = get_element(eq)
    h = get_basis(el, ip.xi)
    #f = el["temperature flux"]
    f = interpolate(el, "temperature flux", ip.xi, t)
    return h*f
end
JuliaFEM.has_rhs(eq::DC2D2) = true

has_rhs (generic function with 2 methods)

In [21]:
el = Seg2([1, 2])
new_fieldset!(el, "geometry")
add_field!(el, "geometry", Field(0.0, Vector[[0.0,0.0], [0.0,1.0]]))
new_fieldset!(el, "temperature flux")
add_field!(el, "temperature flux", Field(0.0, 100.0))
eq = DC2D2(el)
integrate_rhs(eq, 1.0)

2-element Array{Float64,1}:
 50.0
 50.0

## Defining own problem

- main object: takes a set of elements and maps corresponding field equations to them

In [22]:
using JuliaFEM: Problem, get_equation, get_dimension

type PlaneHeatProblem <: Problem
    equations :: Array{Equation, 1}
end
PlaneHeatProblem() = PlaneHeatProblem([])

PlaneHeatProblem

In [23]:
JuliaFEM.get_dimension(pr::Type{PlaneHeatProblem}) = 1
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Quad4}) = DC2D4
JuliaFEM.get_equation(pr::Type{PlaneHeatProblem}, el::Type{Seg2}) = DC2D2

get_equation (generic function with 3 methods)

Our solution procedure so far is therefore

In [24]:
using JuliaFEM: get_connectivity, set_global_dofs!, get_global_dofs
using JuliaFEM: add_element!, get_equations, get_matrix_dimension

# create elements and add necessary properties like connectivity and geometry
el1 = Quad4([2, 3, 4, 5])
new_fieldset!(el1, "geometry")
add_field!(el1, "geometry", Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]))
new_fieldset!(el1, "temperature thermal conductivity")
add_field!(el1, "temperature thermal conductivity", Field(0.0, 6.0))
el2 = Seg2([2, 3])
new_fieldset!(el2, "geometry")
add_field!(el2, "geometry", Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]]))
new_fieldset!(el2, "temperature flux")
add_field!(el2, "temperature flux", Field(1.0, 600.0))

problem = PlaneHeatProblem()
add_element!(problem, el1)
add_element!(problem, el2)

function JuliaFEM.get_connectivity(pr::Problem)
    conn = Int[]
    for eq in get_equations(pr)
        el = get_element(eq)
        append!(conn, get_connectivity(el))
    end
    conn = unique(conn)
    return conn
end

"""
Calculate global dofs for equations, maybe using some bandwidth
minimizing or fill reducing algorithm
"""
function calculate_global_dofs(pr::Problem)
    conn = get_connectivity(pr)
    dim = get_dimension(typeof(pr))
    ndofs = dim*length(conn)
    Logging.debug("total dofs: $ndofs")

    mconn = maximum(conn)
    gdofs = reshape(collect(1:mconn), dim, mconn)
    dofmap = Dict{Int64, Array{Int64, 1}}()
    for (i, c) in enumerate(conn)
        dofmap[c] = gdofs[:, i]
    end
    return dofmap
end

"""
Assign global dofs for equations.
"""
function assign_global_dofs!(pr::Problem, dofmap)
    for eq in get_equations(pr)
        el = get_element(eq)
        c = get_connectivity(el)
        #gdofs = [dofmap[ci] for ci in c]
        gdofs = Int64[]
        for ci in c
            append!(gdofs, dofmap[ci])
        end
        set_global_dofs!(eq, gdofs)
    end
end

dofmap = calculate_global_dofs(problem)
println("dofmap: $dofmap")
assign_global_dofs!(problem, dofmap)

function get_lhs(pr::Problem, t::Float64)
    I = Int64[]
    J = Int64[]
    V = Float64[]
    dim = get_dimension(typeof(pr))
    for eq in filter(has_lhs, get_equations(pr))
        dofs = get_global_dofs(eq)
        lhs = integrate_lhs(eq, t)
        for (li, i) in enumerate(dofs)
            for (lj, j) in enumerate(dofs)
                push!(I, i)
                push!(J, j)
                push!(V, lhs[li, lj])
            end
        end
    end
    return I, J, V
end

function get_rhs(pr::Problem, t::Float64)
    I = Int64[]
    V = Float64[]
    dim = get_dimension(typeof(pr))
    for eq in filter(has_rhs, get_equations(pr))
        dofs = get_global_dofs(eq)
        rhs = integrate_rhs(eq, t)
        for (li, i) in enumerate(dofs)
            push!(I, i)
            push!(V, rhs[li])
        end
    end
    return I, V
end

t = 1.0
A = sparse(get_lhs(problem, t)...)
b = sparsevec(get_rhs(problem, t)..., size(A, 1))

09-Oct 01:41:22:DEBUG:root:total dofs: 4


dofmap: Dict(4=>[3],2=>[1],3=>[2],5=>[4])


4x1 sparse matrix with 2 Float64 entries:
	[1, 1]  =  300.0
	[2, 1]  =  300.0

In [25]:
fdofs = [1, 2]
A[fdofs, fdofs] \ b[fdofs]

2x1 sparse matrix with 2 Float64 entries:
	[1, 1]  =  100.0
	[2, 1]  =  100.0

We still need to consider Dirichlet boundary conditions:
\begin{align}
u &= u_0 && \text{on } \Gamma_{\mathrm{D}} \\
\end{align}

In [26]:
abstract BoundaryProblem <: Problem

type DirichletProblem <: BoundaryProblem
    equations :: Array{Equation, 1}
end
DirichletProblem() = DirichletProblem([])

DirichletProblem

In [27]:
abstract DirichletBC <: Equation

get_unknown_field_name(eq::DirichletBC) = symbol("reaction force")

"""
Dirichlet boundary condition element for 2 node line segment
(plane problems).
"""
type DBC2D2 <: DirichletBC
    element :: Seg2
    integration_points :: Array{IntegrationPoint, 1}
    global_dofs :: Array{Int64, 1}
    fieldval :: Function
end
function DBC2D2(el::Seg2)
    integration_points = [
        IntegrationPoint([-sqrt(1/3)], 1.0),
        IntegrationPoint([+sqrt(1/3)], 1.0)]
    new_fieldset!(el, "reaction force")
    fieldval(X, t) = 0.0
    DBC2D2(el, integration_points, [], fieldval)
end

function JuliaFEM.get_lhs(eq::DBC2D2, ip, t)
    el = get_element(eq)
    h = get_basis(el)(ip.xi)
    return h*h'
end

function JuliaFEM.get_rhs(eq::DBC2D2, ip, t)
    el = get_element(eq)
    h = get_basis(el, ip.xi)
    f = eq.fieldval
    X = interpolate(el, "geometry", ip.xi, t)
    return h*f(X, t)
end
JuliaFEM.has_lhs(eq::DBC2D2) = true
JuliaFEM.has_rhs(eq::DBC2D2) = true
JuliaFEM.get_dimension(pr::Type{DirichletProblem}) = 1
JuliaFEM.get_equation(pr::Type{DirichletProblem}, el::Type{Seg2}) = DBC2D2

get_equation (generic function with 4 methods)

In [28]:
# create elements and add necessary properties like connectivity and geometry
el3 = Seg2([4, 5])
new_fieldset!(el3, "geometry")
add_field!(el3, "geometry", Field(0.0, Vector[[0.0, 0.0], [0.0, 1.0]]))

bc1 = DirichletProblem()
#get_equation(typeof(bc1), typeof(el3))
#DBC2D2(el3)
add_element!(bc1, el3)

1-element Array{JuliaFEM.Equation,1}:
 DBC2D2(JuliaFEM.Seg2([4,5],JuliaFEM.Basis(basis,j),Dict(symbol("reaction force")=>JuliaFEM.Field[],:geometry=>JuliaFEM.Field[JuliaFEM.Field{Array{Array{T,1},1}}(0.0,1,Array{T,1}[[0.0,0.0],[0.0,1.0]])])),[JuliaFEM.IntegrationPoint([-0.5773502691896257],1.0,Dict{Any,Any}()),JuliaFEM.IntegrationPoint([0.5773502691896257],1.0,Dict{Any,Any}())],Int64[],fieldval)

In [29]:
assign_global_dofs!(bc1, dofmap)

# integrate and assembly
t = 1.0
A2 = sparse(get_lhs(bc1, t)...)
b2 = sparsevec(get_rhs(bc1, t)...)

4x1 sparse matrix with 2 Float64 entries:
	[3, 1]  =  0.0
	[4, 1]  =  0.0

Now we have two problems defined, 

In [30]:
full(A)

4x4 Array{Float64,2}:
  4.0  -1.0  -2.0  -1.0
 -1.0   4.0  -1.0  -2.0
 -2.0  -1.0   4.0  -1.0
 -1.0  -2.0  -1.0   4.0

In [31]:
full(b)

4x1 Array{Float64,2}:
 300.0
 300.0
   0.0
   0.0

In [32]:
full(A2)

4x4 Array{Float64,2}:
 0.0  0.0  0.0       0.0     
 0.0  0.0  0.0       0.0     
 0.0  0.0  0.333333  0.166667
 0.0  0.0  0.166667  0.333333

In [33]:
full(b2)

4x1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0

In [34]:
Atot = [A A2; A2 zeros(A2)]

8x8 sparse matrix with 24 Float64 entries:
	[1, 1]  =  4.0
	[2, 1]  =  -1.0
	[3, 1]  =  -2.0
	[4, 1]  =  -1.0
	[1, 2]  =  -1.0
	[2, 2]  =  4.0
	[3, 2]  =  -1.0
	[4, 2]  =  -2.0
	[1, 3]  =  -2.0
	[2, 3]  =  -1.0
	⋮
	[8, 3]  =  0.166667
	[1, 4]  =  -1.0
	[2, 4]  =  -2.0
	[3, 4]  =  -1.0
	[4, 4]  =  4.0
	[7, 4]  =  0.166667
	[8, 4]  =  0.333333
	[3, 7]  =  0.333333
	[4, 7]  =  0.166667
	[3, 8]  =  0.166667
	[4, 8]  =  0.333333

In [35]:
btot = [b; b2]

8x1 sparse matrix with 4 Float64 entries:
	[1, 1]  =  300.0
	[2, 1]  =  300.0
	[7, 1]  =  0.0
	[8, 1]  =  0.0

Problem is that now are total matrix Atot has zero rows which needs to be removed.

In [36]:
full(Atot)

8x8 Array{Float64,2}:
  4.0  -1.0  -2.0       -1.0       0.0  0.0  0.0       0.0     
 -1.0   4.0  -1.0       -2.0       0.0  0.0  0.0       0.0     
 -2.0  -1.0   4.0       -1.0       0.0  0.0  0.333333  0.166667
 -1.0  -2.0  -1.0        4.0       0.0  0.0  0.166667  0.333333
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.333333   0.166667  0.0  0.0  0.0       0.0     
  0.0   0.0   0.166667   0.333333  0.0  0.0  0.0       0.0     

In [37]:
r = unique(rowvals(Atot))
println("Non-zero rows: $r")
xtot = zeros(btot)
F = lufact(Atot[r,r])
s = full(btot[r])
xtot[r] = F \ s
full(xtot)

Non-zero rows: [1,2,3,4,7,8]


8x1 Array{Float64,2}:
 100.0
 100.0
   0.0
   0.0
   0.0
   0.0
 600.0
 600.0

## Developing own solver

Last part. Defining own solver.

- takes a set of problems (typically main problem + boundary problems)
- solves them, updates fields

In [38]:
abstract Solver

"""
Simple solver for educational purposes.
"""
type SimpleSolver <: Solver
    problems
end

"""
Default initializer
"""
function SimpleSolver()
    SimpleSolver(Problem[])
end

function add_problem!(solver::Solver, problem::Problem)
    push!(solver.problems, problem)
end

add_problem! (generic function with 1 method)

In [39]:
get_problems(s::Solver) = s.problems
"""
Call solver to solve a set of problems.

This is simple serial solver for demonstration purposes. It handles the most
common situation, i.e., some main field problem and it's Dirichlet boundary.
"""
function call(solver::SimpleSolver, t)
    problems = get_problems(solver)
    problem1 = problems[1]
    problem2 = problems[2]

    # calculate order of degrees of freedom in global matrix
    # and set the ordering to problems
    dofmap = calculate_global_dofs(problem1)
    assign_global_dofs!(problem1, dofmap)
    assign_global_dofs!(problem2, dofmap)

    # assemble problem 1
    A1 = sparse(get_lhs(problem1, t)...)
    b1 = sparsevec(get_rhs(problem1, t)..., size(A1, 1))

    # assemble problem 2
    A2 = sparse(get_lhs(problem2, t)...)
    b2 = sparsevec(get_rhs(problem2, t)..., size(A2, 1))
    
    # make one monolithic assembly
    A = [A1 A2; A2' zeros(A2)]
    b = [b1; b2]

    # solve problem
    nz = unique(rowvals(A))
    x = zeros(b)
    x[nz] = lufact(A[nz,nz]) \ full(b[nz])

    # get "problem-wise" solution vectors
    x1 = x[1:length(b1)]
    x2 = x[length(b1)+1:end]

    # check residual
    R1 = A1*x1 - b1
    R2 = A2*x2 - b2
    println("Residual norm: $(norm(R1+R2))")

    # update field for elements in problem 1
    for equation in get_equations(problem1)
        gdofs = get_global_dofs(equation)
        element = get_element(equation)
        field_name = get_unknown_field_name(equation)  # field we are solving
        field = Field(t, full(x1[gdofs])[:])
        add_field!(element, field_name, field)
    end

    # update field for elements in problem 2
    for equation in get_equations(problem2)
        gdofs = get_global_dofs(equation)
        element = get_element(equation)
        field_name = get_unknown_field_name(equation)
        field = Field(t, full(x2[gdofs]))
        add_field!(element, field_name, field)
    end
end

call (generic function with 1253 methods)

## Summary

Next we collect all things together and solve Poisson equation in domain $\Omega = [0,1]\times[0,1]$ with some Neumann boundary on $\Gamma_1$ and Dirichlet boundary on $\Gamma_2$.

In [40]:
"""
Simple linspace extension to multidimensional values.
"""
function Base.linspace(X1, X2, n)
    [1/2*(1-ti)*X1 + 1/2*(1+ti)*X2 for ti in linspace(-1, 1, n)]
end

linspace([-1.0, 0.0], [1.0, 2.0], 5)

5-element Array{Array{Float64,1},1}:
 [-1.0,0.0]
 [-0.5,0.5]
 [0.0,1.0] 
 [0.5,1.5] 
 [1.0,2.0] 

In [41]:
using JuliaFEM: get_fieldset

# Define Problem 1:
# - Field function: Laplace equation Δu=0 in Ω={u∈R²|(x,y)∈[0,1]×[0,1]}
# - Neumann boundary on Γ₁={0<=x<=1, y=0}, ∂u/∂n=600 on Γ₁

el1 = Quad4([1, 2, 3, 4])
new_fieldset!(el1, "geometry")
add_field!(el1, "geometry", Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]))
new_fieldset!(el1, "temperature thermal conductivity")
add_field!(el1, "temperature thermal conductivity", Field(0.0, 6.0))

el2 = Seg2([1, 2])
new_fieldset!(el2, "geometry")
add_field!(el2, "geometry", Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0]]))

# Boundary load, linear ramp 0 -> 600 at time 0 -> 1
new_fieldset!(el2, "temperature flux")
add_field!(el2, "temperature flux", Field(0.0, 0.0))
add_field!(el2, "temperature flux", Field(1.0, 600.0))

problem1 = PlaneHeatProblem()
add_element!(problem1, el1)
add_element!(problem1, el2)

# Define Problem 2:
# - Dirichlet boundary Γ₂={0<=x<=1, y=1}, u=0 on Γ₂

el3 = Seg2([3, 4])
new_fieldset!(el3, "geometry")
add_field!(el3, "geometry", Field(0.0, Vector[[1.0, 1.0], [0.0, 1.0]]))

problem2 = DirichletProblem()
add_element!(problem2, el3)

# Create a solver for a set of problems
solver = SimpleSolver()
add_problem!(solver, problem1)
add_problem!(solver, problem2)

# Solve problem at time t=1.0 and update fields
call(solver, 1.0)

# Postprocess.
# Interpolate temperature field along boundary of Γ₁ at time t=1.0
xi = linspace([-1.0], [1.0], 5)
X = interpolate(el2, "geometry", xi, 1.0)
T = interpolate(el2, "temperature", xi, 1.0)
println(X)
println(T)

09-Oct 01:41:25:DEBUG:root:total dofs: 4


Residual norm: 9.845568954283847e-14
Array{T,1}[[0.0,0.0],[0.25,0.0],[0.5,0.0],[0.75,0.0],[1.0,0.0]]
[100.00000000000003,100.00000000000003,100.00000000000003,100.00000000000003,100.00000000000003]


## TODO

- dynamics
- different basis for trial and test (Petrov-Galerkin)

## Frequently asked questions

Any question about data structures, better ideas and improvements are very welcome.